In [46]:
%pip install nltk


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ralva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [47]:
# Lectura de dades (Movie Reviews Corpus) NLTK

import nltk
import re
import sklearn
from sklearn.model_selection import train_test_split 
nltk.download('movie_reviews')
nltk.download('wordnet')
from nltk.corpus import movie_reviews as mr
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
all_texts = []
all_categories = []

for fileid in mr.fileids():
    all_texts.append(mr.raw(fileid))
    all_categories.append(mr.categories(fileid)[0])
    


In [49]:
all_categories.count('neg')

1000

In [50]:
# Preprocessing text
def preprocessing(text):
  # Remove digits
  for num in range(10):
    text = text.replace(str(num), "")

  # Pasar tot el text a mínuscules
  text = text.lower()

  # Substituir dos espais per un
  text = re.sub(" +", " ", text)

  # Concatenar frases amb dos espais
  text = text.replace("\n ", "\n  ")

  #Treure tabs
  text = text.replace('\t','')
  
  # Eliminem stopwords al countvectorizer
  
  # Lematitzar text
  lemmatizer = WordNetLemmatizer()
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
  
  return text

In [51]:
for text in all_texts:
  text = preprocessing(text)

In [52]:
# Split data into train, test and validation
X_train, X_test, y_train, y_test = train_test_split(all_texts, all_categories, test_size=0.2, random_state=14)

In [53]:
# Count vectorizer

# Crear un objeto CountVectorizer
vectorizer = CountVectorizer(stop_words=stopwords.words('english'), min_df=5)

# Ajustar el vectorizador a los datos de entrenamiento
train_matrix = vectorizer.fit_transform(X_train)

# Transformar los datos de validación y test
test_matrix = vectorizer.transform(X_test)

In [62]:
#Hyperparams
n_estimators = [100, 200, 500]
max_depth = [3, 5, 10]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2]

In [63]:
# Grid search for hyperparameter tuning
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

rf = RandomForestClassifier()
rf_cv = GridSearchCV(rf, param_grid, cv=5)
rf_cv.fit(train_matrix, y_train)
print("Tuned Random Forest Parameters: {}".format(rf_cv.best_params_))

Tuned Random Forest Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}


In [64]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

# Crear un objeto RandomForestClassifier para saber si el texto es positivo o negativo
classifier = RandomForestClassifier(random_state=14, n_estimators=rf_cv.best_params_['n_estimators'], max_depth=rf_cv.best_params_['max_depth'], min_samples_split=rf_cv.best_params_['min_samples_split'], min_samples_leaf=rf_cv.best_params_['min_samples_leaf'])

# Entrenar el clasificador
classifier.fit(train_matrix, y_train)

# Predecir las categorías de los datos de validación
test_predictions = classifier.predict(test_matrix)

In [57]:
for i, prediction in enumerate(test_predictions):
    print(f"Prediction: {prediction}")
    print(f"Real_category: {y_test[i]}")
    print("-----------------------------")
    print(f"Text: {X_test[i]}")

Prediction: pos
Real_category: pos
-----------------------------
Text: when you go to the movies as much as i do , you unfortunately end up seeing certain movie trailers one too many times . 
such was the case with frequency . 
every time i went to a screening , there was the frequency preview . 
it looked awful . 
it looked cheesy , sappy and ridiculous . 
it looked like a flop . 
well , whoever put that trailer together should be fired , because in terms of pure entertainment , frequency is one of the best of the year thus far . 
frequency boasts a compelling story line . 
an occurrence of freakish solar activity allows police detective john sullivan ( james caviezel ) to speak to his fireman father frank sullivan ( dennis quaid ) through a ham radio , despite the fact that frank has been dead for 30 years . 
john is able to give his father information that prevents his death in a warehouse fire , but by doing so causes other changes . 
somehow , a serial killer's reign of terror , w

In [65]:
# Calcular la precisión
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, test_predictions)

accuracy

0.86

In [59]:
def predict_sentiment(text):
  text = preprocessing(text)
  text_matrix = vectorizer.transform([text])
  prediction = classifier.predict(text_matrix)
  return prediction[0]

In [71]:
predict_sentiment("""A world, by the way, systemic, where the physics and artificial intelligence of the characters and enemies generate a lot of spontaneous moments. Especially in the combats, constant and hilarious, especially when giant monsters are involved; and very customizable thanks to the great vocation and pawn systems. We must point out the performance flaws, a somewhat limited bestiary and a visual section with its lights and shadows, but in the end they are minor details in a title that has made us say out loud several times "I can't believe it." Such a surprising title that teaches you to expect the unexpected.""")

'pos'

In [67]:
# SVM classifier
from sklearn.svm import SVC

SVC_classifier = SVC(random_state=14)

SVC_classifier.fit(train_matrix, y_train)

SVC_val_predictions = SVC_classifier.predict(test_matrix)

accuracy_score(y_test, SVC_val_predictions)

0.7775